On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. Translated 32% survival rate.
One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew.
Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this notebook we will go through some simple analysis and refer to the tools that you have seen in the Datacamp courses. We will train a couple of simple models to predict the survival rate with machine learning. First, let's list the data that are available on this kaggle notebook

In [ ]:
import os
for dirname, _, filenames in os.walk('data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In the input folder we find the train and test data in csv format.

Please install the requirements mentioned in the `requirements.txt` file before continuing.

# Data exploration and cleaning


As a first step, we will explore the distribution of data, their correlation and type. First, let's import the the data as a pandas dataframe.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_df = pd.read_csv('data/train.csv')

In [ ]:
train_df.head()

In the dataframe, each column represents:
* survival -> Survival -> 0 = No, 1 = Yes
* pclass -> Ticket class -> 1 = 1st, 2 = 2nd, 3 = 3rd
* sex -> Sex -> male/female
* Age -> Age in years
* sibsp -> # of siblings / spouses aboard the Titanic
* parch -> # of parents / children aboard the Titanic
* ticket -> Ticket number
* fare -> Passenger fare
* cabin -> Cabin number
* embarked -> Port of Embarkation -> C = Cherbourg, Q = Queenstown, S = Southampton

With `.info()` we check numerical/categorical columns

In [ ]:
train_df.info()

# Cleaning categorical features

To get some info on non-numerical variables, we can still use the `.describe()` command but we need to pass the argument `include=['object']`

In [ ]:
train_df.describe(include=['object'])

In [ ]:
train_df[['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']].head()

Some important remarks:
* Names are unique across the dataset (count=unique=891)
* Sex variable has two possible values with 65% male (top=male, freq=577/count=891).
* Cabin values have several dupicates across samples. In other words, several passengers shared a cabin.
* Embarked takes three possible values. S port used by most passengers (top=S)
* Ticket feature has high ratio (22%) of duplicate values (unique=681).

**Task: 1**

Which column has the most number of missing values?

In [ ]:
### Your solution here ###

Most missing values in column _____

---

To simplify our analysis, we will drop the 'Ticket', 'Cabin' and 'Embarked' columns.

In [ ]:
train_df = train_df.drop(['Ticket', 'Cabin', 'Embarked'], axis=1)

### Feature Engineering: Creating new features from existing ones: Titles from Names

In [ ]:
train_df.head(10)

We see that the names contains information about the titles of the passengers. Lets extract the titles from the names.

**Task: 2**

You can use regular expression to extract the titles. What should we be looking for? How does the title stand apart from the rest of the name?

Once we have the regex expression, we can use `str.extract()` method with the regex expression on the Name column.

Extract the titles to a new column named `'Title'`.

In [ ]:
### Your solution here ###

The frequency of occurance of each of the titles in males and females can be obtained by using a cross tabulation frequency table.



In [ ]:
pd.crosstab(train_df['Title'], train_df['Sex'])

We see that there are several titles, some referring to the same type of person. Some of the title are very rare among the passengers.

**Task: 3**

* Replace the following titles with the word `'Rare'`.


`['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer']`

* Replace the title `'Mlle'` with `'Miss'`. (Mademoiselle is a French courtesy title, abbreviated Mlle, traditionally given to an unmarried woman)

* Replace the title `'Mme'` with `'Mrs'`. (French title Madame for married woman)

* Replace the title `'Ms'` with `'Miss'`. 



In [ ]:
### Your solution here ###

We should now have 5 titles. Let us check the survival rate of each of these title holders.


**Task: 4**

Calculate the mean value of the column `'Survived'` for each of the titles. 

In [ ]:
### Your solution here ###

The most survival rate was associated with the title _____

---

### Categorical value encoding 

It is easier to work with numeric features. We can convert the `Sex` and `Title` columns to numeric columns.

**Task: 5**

Convert the `Sex` column to numeric column `is_female` with value 1 for female and 0 for male.

In [ ]:
### Your solution here ###

In [ ]:
train_df.head()

It is often useful to encode categorical variables into one-hot encodings. We can convert the `Title` column to one-hot encoding of the constituent values.

**Task: 6**

Convert the column `Title` to one-hot encoding and add to the dataframe. Then new column names should be like `'is_Mr'`, `'is_Miss'`, etc

In [ ]:
### Your solution here ###

In [ ]:
train_df.head()

At this point, we can drop the `'Name'`, `'Sex'` and `'Title'` columnns

In [ ]:
train_df.drop(['Name', 'Sex', 'Title'] , axis=1, inplace = True)

In [ ]:
train_df.head()

# Cleaning Numerical features

Now that we have dealt with categorical columns in our dataframe, we can move on to analysing the numerical columns.

### Feature engineering

We can associate for each passenger the number of family members. From this we can also get a new feature representing if the passenger is alone or not.

**Task: 7**

Create a new column named `'FamilySize'`. The value should be the number of siblings/ spouse + the number of parents/ children + the person itself.

Also calculate the survival rate for each of the family size.

In [ ]:
### Your solution here ###

The survival rate was maximum for the family size _____

---

**Task: 8**

Another similar feature we can extract is whether a person was alone or not. Add a new column is_alone with value 1 if the passenger is alone and 0 if he/she has family members together with him/her.

Calculate the survival rate for passengers who are not alone.

In [ ]:
### Your solution here ###

Survival rate for passengers who are not alone is _____


---



Now we can drop the columns `SibSp` and `Parch`. 

In [ ]:
train_df.drop(['SibSp', 'Parch'] , axis=1, inplace = True)

We can visualize the correlations between features by plotting a heatmap of the correlation values.

In [ ]:
corr_matr = train_df.corr()
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr_matr, cmap = cmap)

From the plots, we have seen that the `'PassengerId'` column has almost no correlation with the other columns. It is just a high cardinality feature. Therefore, we will drop it from the training dataframe.

In [ ]:
train_df = train_df.drop(['PassengerId'], axis=1)

**Conclusions from data analysis and feature extraction**

**Dropped features**

* Ticket feature was be dropped from our analysis as it contains high ratio of duplicates (22%) and there may not be a correlation between Ticket and survival.
* Cabin feature was be dropped as it is highly incomplete or contains many null values.
* PassengerId was be dropped from training dataset as it not correlated to survival.


**Created features**

* Created a new feature called Family based on Parch and SibSp to get total count of family members on board.
* Extracted Title from Name as a new feature. One-hot encoded it for modelling.

# Model training and testing



We haven't yet seen how to develop models and train it on the data we have. These will be covered in the future datacamp courses. This is still left here for completion. 

Let's create the proper train and validation set from the train_df dataframe.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


Create the features and labels from the data and split the data into training set (70%) and validation set (30%). 

In [ ]:
X = train_df.loc[:, train_df.columns != 'Survived']
y = train_df.loc[:, 'Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

**Question**

We are now going to train 4 different models, 2 Knn (one scaled, one not) and 2 Random Forest models (also, one scaled, one not). What to we expect from scaling the input data?

a) Knn score improves, Random forest remains (more or less) the same \
b) Both Knn and Random forest scores improve \
c) Both scores remain the same (more or less) \
d) Both scores get worse

# KNN training and scoring

In sklearn, both Knn and Random Forests do not handle missing values internally. Therefore, we need to impute the missing values before passing it through the models

In [ ]:
X_train = X_train.fillna(X_train.mean())
X_val= X_val.fillna(X_val.mean())

We can now move on to training the KNN model

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
knn.score(X_val, y_val)

Now train the model on scaled data.

In [ ]:
ss = StandardScaler()

X_train_scaled = ss.fit_transform(X_train)
X_val_scaled = ss.transform(X_val)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train_scaled, y_train)
knn.score(X_val_scaled, y_val)

In this example we have only used a standard scaler, but often other types of scaling yield better results. For example, some features may benefit from log scaling their values (typical for "fat tails")

In [ ]:
sns.pairplot(train_df[['Fare', 'Age']])

# Random forest training and scoring

Train a Random Forest classifier on the unscaled data.

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
print(f'accuracy score: {accuracy_score(rf.predict(X_val), y_val)}')
print(f'Feature importance: \n {dict(zip(X.columns, rf.feature_importances_.round(2)))}')

Now train a Random Forest classifier on the scaled data.

In [ ]:
rf.fit(X_train_scaled, y_train)
print(f'accuracy score: {accuracy_score(rf.predict(X_val_scaled), y_val)}')
print(f'Feature importance: \n {dict(zip(X.columns, rf.feature_importances_.round(2)))}')

Notice that a feature being correlated to survival does not necessarily mean that the feature importance in a model is going to be high

In [ ]:
corr_matr = train_df.corr()
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr_matr, cmap = cmap)